##### import libs

In [1]:
import numpy as np
import pandas as pd
import implicit
import matplotlib.pyplot as plt
from tqdm import tqdm
from pathlib import Path
from scipy.sparse import coo_matrix, csr_matrix
from sklearn.model_selection import train_test_split

/home/sokolov/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from utils import evaluate, load_data

##### read data

In [3]:
user_item_data, user_meta_data, item_meta_data, test_pairs_data = load_data()

##### split data

In [4]:
ui_train, ui_val = train_test_split(user_item_data,
                                    test_size=0.15,
                                    random_state=42,
                                    shuffle=False)

In [5]:
u_train = ui_train.user_id
i_train = ui_train.item_id
likes_train = ui_train.like
dislikes_train = ui_train.dislike

u_val = ui_val.user_id
i_val = ui_val.item_id
likes_val = ui_val.like
dislikes_val = ui_val.dislike

In [7]:
sparse_train = coo_matrix((likes_train-dislikes_train, (u_train, i_train)))
sparse_val = coo_matrix((likes_val-dislikes_val, (u_val, i_val)))

##### als model

In [8]:
model = implicit.als.AlternatingLeastSquares(factors=256)
model.fit(sparse_train)

/home/sokolov/workspace/vk_recsys/vk_env/lib/python3.10/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 4.219879388809204 seconds
  warnings.warn(
100%|██████████| 15/15 [01:22<00:00,  5.47s/it]


##### evaluation

In [ ]:
model = model.to_cpu()

/home/sokolov/workspace/vk_recsys/vk_env/lib/python3.10/site-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 16 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


: 

In [ ]:
als_score = (model.user_factors[ui_val['user_id']] * 
             model.item_factors[ui_val['item_id']]).sum(axis=1)

In [ ]:
ui_val['als_score'] = als_score

In [ ]:
ui_val["target"] = ui_val["like"].astype(np.int8) - ui_val["dislike"].astype(np.int8)

In [ ]:
evaluate(user_id=ui_val.user_id.values,
         target=ui_val.target.values,
         score=ui_val.als_score.values)

100%|██████████| 181721/181721 [00:52<00:00, 3467.31it/s]


np.float64(0.6178085542103173)

##### submission

In [ ]:
test_als_score = (model.user_factors[test_pairs_data['user_id']] * 
                  model.item_factors[test_pairs_data['item_id']]).sum(axis=1)

In [ ]:
test_pairs_data["predict"] = test_als_score
test_pairs_data.to_csv("./submission.csv",index=False)